<a href="https://colab.research.google.com/github/KholiswaT/AmazonVineReviews/blob/main/AmazonVineReviews_Beauty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 htt

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-01-23 02:27:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-01-23 02:27:50 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AWS").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [6]:
from pyspark import SparkFiles
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", inferSchema=True, header=True)

# Count Number of Review in this dataset (US Beauty)
df.count()

5115666

In [7]:
#Show Dataframe
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [8]:
#Only columns that matched the Schema SQL file were selected. The alias funciton is used to hid the date format --without this, the date format will be shown in the column alongside review_date
review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias('review_date')])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612| 2015-08-31|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691| 2015-08-31|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471| 2015-08-31|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889| 2015-08-31|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747| 2015-08-31|
|R1R30FA4RB5P54|    2909389|B000NYL1Z6|     166146615| 2015-08-31|
|R30IJKCGJBGPJH|   19397215|B001SYWTFG|     111742328| 2015-08-31|
|R18GLJJPVQ1OVH|    3195210|B005F2EVMQ|     255803087| 2015-08-31|
| R8TVYIJXLYJT0|   52216383|B00M1SUW7K|     246816549| 2015-08-31|
|R1CJGF6M3PVHEZ|   10278216|B001KYQA1S|       9612905| 2015-08-31|
|R2A30ALEGLMCGN|   24655453|B00SAQ9DZY|     292127037| 2015-08-31|
|R134GP1Y3EQZ1L|   30788223|B00HFQQ0VU|     311309827| 2015-08

In [9]:
products = df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001ANOOOE|The Naked Bee Vit...|
|B0016J22EQ|Alba Botanica Sun...|
|B00HU6UQAG|Elysee Infusion S...|
|B002HWS7RM|Diane D722 Color,...|
|B00SM99KWU|Biore UV Aqua Ric...|
|B000NYL1Z6|Murad Clarifying ...|
|B001SYWTFG|CoverGirl Queen C...|
|B005F2EVMQ|Bifesta Mandom Ey...|
|B00M1SUW7K|Can You Handlebar...|
|B001KYQA1S|Maybelline Great ...|
|B00SAQ9DZY|12 New, High Qual...|
|B00HFQQ0VU|Waterpik Aquarius...|
|B00PYL8MAA|Proganix Agave Ne...|
|B00D9NV2D4|         Shea Butter|
|B008Y9M412|Etude House Fresh...|
|B00FWXBLHG|Vintage Lil' Spon...|
|B003KL8CB0|Jenna Jameson Hea...|
|B006R2LNV0|Korean Cosmetics_...|
|B00SMIIR0W|Schick Hydro Silk...|
|B003UKKH6O|Oral-B Vitality P...|
+----------+--------------------+
only showing top 20 rows



In [10]:
#Group by the Customer ID to aggregate the data and count then rename the column to match the SQL table 
customers = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9015608|             3|
|    9767880|             2|
|   15958313|             1|
|   12879980|            20|
|   43920023|            31|
|     106535|             1|
|   20328662|             1|
|   12819130|             4|
|   50145309|             1|
|   15159322|             2|
|   16747249|             2|
|   37678162|             1|
|     227763|             1|
|   15191144|             2|
|   31946913|             1|
|   44295916|             1|
|   19021463|             1|
|     806337|             2|
|   16306618|            18|
|     393186|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
vine = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3I2DHQBR577SS|          5|            0|          0|   N|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|
| RAI2OIG50KZ43|          5|            0|          0|   N|
|R1R30FA4RB5P54|          4|            0|          0|   N|
|R30IJKCGJBGPJH|          5|            0|          0|   N|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|
| R8TVYIJXLYJT0|          5|            0|          0|   N|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|
|R2A30ALEGLMCGN|          4|            1|          1|   N|
|R134GP1Y3EQZ1L|          5|            0|          0|   N|
| R6CE3SOIUJGP4|          4|            1|          1|   N|
|R37CDLQZUPDZVM|          5|            

In [12]:
mode = "append"
RDS_url = "jdbc:postgresql://awvine-db.cgcpr4ffz7uv.us-east-2.rds.amazonaws.com:5432/awvineclass_db"
config = {"user":"root", "password": "password", "driver":"org.postgresql.Driver"}

In [18]:
review_id.write.jdbc(url=RDS_url, table='aws_beauty.review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=RDS_url, table='aws_beauty.products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=RDS_url, table='aws_beauty.customers', mode=mode, properties=config)

In [ ]:
vine.write.jdbc(url=RDS_url, table='aws_beauty.vine_table', mode=mode, properties=config)